In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from AlgoTestFactory import *

# altf1 = AlgoTestFactory([0, 10000], 15, savePath="Combined_4_test2", storeAccuracies=True)

# altf1.run()

In [3]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json
import pprint
import csv
from utilities import *
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [4]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

goodFileDict = pickle.load(open('mergedGood_v3.p', 'rb'))
algo = SktWsegRWR(
            w2w_modelFunc = pb.get_w2w_mat, 
            t2t_modelFunc = pb.get_cng2cng_mat,
            v2c_modelFunc = pb.get_v2c_ranking,
            sameCng_modelFunc = pb.get_w2w_samecng_mat,
            partition=[0.25, 0.25, 0.25, 0.1]
        )

In [91]:
np.set_printoptions(precision=4)
# for f in list(goodFileDict.keys())[1:20]:
# for f in list(goodFileDict.keys())[1:100]:
for f in fs:
#     print('=='*20)
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj, verbose = False)
        if(result != None):
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
            ac = Accuracy(result, dcsObj)
            print(f, "\t", ac)

334655.p 	 50.0
434538.p 	 33.333333333333336


In [143]:
f = '292204.p'
s, d = loadSentence(f, goodFileDict[f])
SeeSentence(s)

SKT ANALYZE
---------------
AkarzaPalakaM dyUtaPalakaM ca   
Analyzing: AkarzaPalakam
0 : Akarza [0, 'Akarza', 3] 
1 : karza [1, 'kfz', -32] 
10 : kam [2, 'ka', 69] 
6 : Palakam [3, 'Palaka', 31] [4, 'Palaka', 71] 
6 : Pala [5, 'Pal', -32] 
-------------------------
Analyzing: dyUtaPalakam
0 : dyUta [6, 'dyUta', 3] [7, 'dIv', -190] 
0 : dyu [8, 'dyu', 3] 
0 : dyU [9, 'dyu', 74] [10, 'dyu', 34] 
9 : kam [11, 'ka', 69] 
2 : uta [12, 'uta', 3] [13, 'vA', -190] 
2 : Uta [14, 'Uta', 3] 
2 : uta [15, 'uta', 2] 
5 : Palakam [16, 'Palaka', 31] [17, 'Palaka', 71] 
5 : Pala [18, 'Pal', -32] 
-------------------------
Analyzing: ca
0 : ca [19, 'ca', 2] 
-------------------------
Analyzing  AkarzaPalakam
0 :  Akarza ['Akarza'] [{'compound': ['iic.']}]
0 :  A ['A'] []
1 :  karza ['kfz'] [{'verb': ['imp. [1] ac. sg. 2']}]
10 :  kam ['ka'] [{'noun': ['acc. sg. m.']}]
6 :  Palakam ['Palaka'] [{'noun': ['acc. sg. n.', 'nom. sg. n.']}]
6 :  Pala ['Pal'] [{'verb': ['imp. [1] ac. sg. 2']}]
Analyzing  dyUt

In [142]:
fs = [ '292204.p']
def debugSentences(fs):
    np.set_printoptions(precision=4)
    # for f in list(goodFileDict.keys())[1:20]:
    # for f in list(goodFileDict.keys())[1:100]:
    for f in fs:
    #     print('=='*20)
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        if(sentenceObj != None):
            result, detail = algo.predict(sentenceObj, dcsObj, verbose = True)
            if(result != None):
                solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
                ac = Accuracy(result, dcsObj)
                print(f, "\t", ac)
                display(detail['sentence'])
                display(detail['DCSLemmas'])
                display(result)
#                 print(detail['nodeList'])
                print('Initial Query:', detail['initialQuery'])
                for step in range(detail['steps']):
                    print('-'*30)
                    print('Step:', step)
                    print('-'*30)
                    print('Winner:', detail[str(step)]['winner'])
                    print('Removed:', detail[str(step)]['removed'])
                    print('Query:', detail[str(step)]['updated_query'])
        print('='*30)

debugSentences(fs)

292204.p 	 40.0


'AkarzaPalakaM dyUtaPalakaM ca   '

[['Akarza', 'Palaka'], ['dyUta', 'Palaka'], ['ca']]

['ca', 'vA', 'ka', 'Pal', 'ka', 'Pal', 'Akarza', 'dyu']

Initial Query: [19]
------------------------------
Step: 0
------------------------------
Winner: (13, 'uta', 'vA', -190)
Removed: [(12, 'uta', 'uta', 3), (6, 'dyUta', 'dyUta', 3), (7, 'dyUta', 'dIv', -190), (14, 'Uta', 'Uta', 3), (15, 'uta', 'uta', 2)]
Query: [19, 13]
------------------------------
Step: 1
------------------------------
Winner: (11, 'kam', 'ka', 69)
Removed: [(16, 'Palakam', 'Palaka', 31), (17, 'Palakam', 'Palaka', 71)]
Query: [19, 13, 11, 18]
------------------------------
Step: 2
------------------------------
Winner: (2, 'kam', 'ka', 69)
Removed: [(3, 'Palakam', 'Palaka', 31), (4, 'Palakam', 'Palaka', 71)]
Query: [19, 13, 11, 18, 2, 5]
------------------------------
Step: 3
------------------------------
Winner: (0, 'Akarza', 'Akarza', 3)
Removed: [(1, 'karza', 'kfz', -32)]
Query: [19, 13, 11, 18, 2, 5, 0]


In [13]:
with open('.temp/sandhi_encounters.csv', 'w') as fh:
    fcsv = csv.writer(fh)
    fcsv.writerow(['left', 'right', 'derivations', 'word_left', 'word_right', 'pos_left', 'pos_right'])
np.set_printoptions(precision=6)
# for f in list(goodFileDict.keys())[90:91]:
for f in list(goodFileDict.keys())[1:100]:
# for f in fullfs:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        if(result != None):
            ac = Accuracy(result, dcsObj)
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#             print(result)
#             print('vs')
#             print(solution)
#             print(f, "\t", ac)

In [31]:
# fullfs = ['105587.p','293473.p','379245.p']
fullfs = ['302864.p', '292204.p', '161425.p', '128533.p', '265112.p', '16349.p', '320178.p', '334655.p', '331468.p', '365975.p', '434538.p', '148267.p', '340997.p', '204751.p', '289589.p', '423330.p', '276728.p', '105056.p', '325962.p', '383515.p', '60055.p', '228316.p', '192549.p', '169215.p', '393452.p', '433996.p', '440103.p', '68686.p', '157189.p', '440567.p']

In [200]:
fullfs = ['302864.p', '292204.p', '161425.p', '128533.p', '265112.p', '16349.p', '320178.p', '334655.p', '331468.p', '365975.p', '434538.p', '148267.p', '340997.p', '204751.p', '289589.p', '423330.p', '276728.p', '105056.p', '325962.p', '383515.p', '60055.p', '228316.p', '192549.p', '169215.p', '393452.p', '433996.p', '440103.p', '68686.p', '157189.p', '440567.p']
def GetSomeJSON():
    lowScore = []
    midScore = []
    highScore = []
    np.set_printoptions(precision=4)
#     for i in range(1, 400):
#         f = list(goodFileDict.keys())[i]
#     print(len(fullfs))
    for f in fullfs:
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        if(sentenceObj != None):
            result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)

            if(result != None):
                runDetails['file'] = f
                ac = runDetails['accuracy']
#                 print(f, ac)

                # CODE FOR SELECTING FILES 
#                 if(i%100 == 0):
#                     print('Chekpoint:', i)
                if(ac <= 40):
                    lowScore.append(runDetails)
                elif (ac <= 60):
                    if(ac >= 45):
                        midScore.append(runDetails)
                elif (ac <= 100):
                    if(ac >= 80):
                        highScore.append(runDetails)

    print('Lowscore files found:', len(lowScore))
    print('Midscore files found:', len(midScore))
    print('Highscore files found:', len(highScore))

    lowScore = lowScore[0:10]
    midScore = midScore[0:10]
    highScore = highScore[0:10]
    pickle.dump(lowScore, open('lowScore.p', 'wb'))
    pickle.dump(midScore, open('midScore.p', 'wb'))
    pickle.dump(highScore, open('highScore.p', 'wb'))

#=================================================================
GetSomeJSON()

Lowscore files found: 4
Midscore files found: 7
Highscore files found: 9


In [5]:
def MacroAccuracy():
    allAcs = []
    np.set_printoptions(precision=4)
    for i in range(1, 5000):
        f = list(goodFileDict.keys())[i]
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        if(sentenceObj != None):
            result = algo.predict(sentenceObj, dcsObj, verbose = False)

            if(result != None):
                ac = Accuracy(result, dcsObj)
                allAcs.append(ac)

                # CODE FOR SELECTING FILES 
                if(i%100 == 0):
                    print('Chekpoint:', i)
    allAcs = np.array(allAcs)
    return allAcs


#=================================================================
allAcs = MacroAccuracy()

In [6]:

# the histogram of the data
n, bins, patches = plt.hist(allAcs, 40, normed=False)

plt.xlabel('MicroAccuracy')
plt.ylabel('Count')
plt.title('Histogram of MicroAccuracy')
plt.grid(True)
plt.show()

print('Macro Accuracy %:', 100*np.sum(n[(bins[::] > 95)[:-1:]])/np.sum(n))

Macro Accuracy %: 37.5
[  66.6667   67.5      68.3333   69.1667   70.       70.8333   71.6667
   72.5      73.3333   74.1667   75.       75.8333   76.6667   77.5
   78.3333   79.1667   80.       80.8333   81.6667   82.5      83.3333
   84.1667   85.       85.8333   86.6667   87.5      88.3333   89.1667
   90.       90.8333   91.6667   92.5      93.3333   94.1667   95.
   95.8333   96.6667   97.5      98.3333   99.1667  100.    ]


In [ ]:
def SeeSomeJSON(region = 'mid', files = ['lowScore.p', 'midScore.p', 'highScore.p']):
    np.set_printoptions(precision=3)
    if region == 'low':
        p = pickleFixLoad(files[0])
    elif region == 'mid':
        p = pickleFixLoad(files[1])
    else:
        p = pickleFixLoad(files[2])
#     for pf in p:
#         print(pf['file'])
    n_steps = p[0]['steps']
    for s in range(n_steps):
        
        print('Step:', s)
        print('='*20)
        
        for key, val in p[0][str(s)].items():
            print(key, ":")
            print(val)
            print('-'*20)
        
SeeSomeJSON('low')
print('='*20)
SeeSomeJSON('mid')
print('='*20)
SeeSomeJSON('high')

In [68]:
def ArePredictionsOK():
    with open('.temp/weird.csv', 'w') as fh:
        wcsv = csv.writer(fh)
        wcsv.writerow(['filename', 'sentence', 'solution', 'prediction'])
        np.set_printoptions(precision=4)
        badCount = 0
        for i in range(1, 600):
            f = list(goodFileDict.keys())[i]

            sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
            if(sentenceObj != None):
                result = algo.predict(sentenceObj, dcsObj, verbose = False)
                if(result != None):            
                    solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
                    ac = Accuracy(result, dcsObj)
                    if ac >= 100 and len(result) != len(solution):
                        badCount += 1
                        if(badCount%10 == 0):
                            print("="*40)
                        sline = [f, sentenceObj.sentence, solution, result]
                        wcsv.writerow(sline)
        print(badCount, 'weird predictions encountered.')
        
ArePredictionsOK()

35 weird predictions encountered.


In [7]:
SeeSentence(sentenceObj)

SKT ANALYZE
---------------
upacAreRa SIlena rUpayOvanasaMpadA   
Analyzing  upacAreRa
0 :  upacAreRa ['upacAra'] [{'noun': ['i. sg. m.']}]
Analyzing  SIlena
0 :  SIlena ['SIla'] [{'noun': ['i. sg. m.', 'i. sg. n.']}]
0 :  SIla ['SIla'] [{'compound': ['iic.']}]
0 :  SIle ['SIla'] [{'noun': ['loc. sg. m.', 'acc. du. n.', 'nom. du. n.', 'loc. sg. n.']}]
0 :  SIla ['SIl'] [{'verb': ['imp. [1] ac. sg. 2']}]
3 :  ina ['na'] [{'noun': ['voc. sg. m.', 'voc. sg. n.']}]
4 :  na ['na'] [{'indeclinable': ['part.']}]
Analyzing  rUpayOvanasampadA
0 :  rUpa ['rUpa'] [{'compound': ['iic.']}]
10 :  sampadA ['sampad_2'] [{'noun': ['i. sg. f.']}]
4 :  yOvana ['yOvana'] [{'compound': ['iic.']}]
4 :  yO ['yu_3'] [{'noun': ['loc. sg. m.', 'loc. sg. f.']}]
6 :  vana ['vana'] [{'compound': ['iic.']}]
6 :  vana ['van'] [{'verb': ['imp. [1] ac. sg. 2']}]



In [8]:
SeeDCS(dcsObj)

DCS ANALYZE
---------------
upacAreRa SIlena rUpayOvanasaMpadA   
[['upacāra'], ['śīla'], ['rūpa', 'yauvana', 'sampad']]
Lemmas: ['upacAra', 'SIla', 'rUpa', 'yOvana', 'sampad']
[['89'], ['91'], ['3', '3', '90']]



In [7]:
def CSVforSupervised():
    np.set_printoptions(precision=4)
    # Clean the csv files
    metPerfFH = open('.temp/metPerfFH.csv', 'w')
    metPerfFH_bin = open('.temp/metPerfFH_bin.csv', 'w')

    metPerfFH.write('w2w,t2t,w2w_samecng,InDCS\n')
    metPerfFH_bin.write('w2w,t2t,w2w_samecng,InDCS\n')

    metPerfFH.close()
    metPerfFH_bin.close()
    ######################
    gfl = list(goodFileDict.keys())
    for i in range(0, 11000):
        f = gfl[i]
        if(i%250 == 0):
            print('Checkpoint:', i)
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        
        if(sentenceObj != None):
            result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)




In [ ]:
np.set_printoptions(precision=6)
jsonBatch = []
for f_ in lowScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/lowScore.p','wb'))
print('33 % Complete')
#################################################################
jsonBatch = []
for f_ in midScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/midScore.p','wb'))
print('67 % Complete')
#################################################################
jsonBatch = []
for f_ in highScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
print(pprint.pprint(jsonBatch))
pickle.dump(jsonBatch, open('.temp/highScore.p','wb'))
print('100 % Complete')

In [195]:
a = np.array([1,2,3,4,5])
a[:-1:]

array([1, 2, 3, 4])